Fluent Python Chapter 14读书笔记

#### 可迭代对象

> An **iterable** is an object that has an `__iter__` method which returns an iterator, or which defines a `__getitem__` method that can take sequential indexes starting from zero. So an iterable is an object that you can get an iterator from.

Python中的对象，只要它定义了可以返回一个迭代器的`__iter__`方法，或者定义了可以支持下标索引的`__getitem__`方法，那么它就是一个可迭代对象(Iterable)。简单说，可迭代对象就是能提供迭代器的任意对象。

It’s important to be clear about the relationship between iterables and iterators: Python obtains iterators from iterables.

The most accurate way to check whether an object $x$ is iterable is to call `iter(x)` and handle a `TypeError` exception if it isn’t.

In [4]:
import re
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = re.findall(r'w+', text)
        
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % self.text

def isIterable(x):
    try:
        iter(x)
    except TypeError:
        return False
    else:
        return True
sentence = Sentence("hello")
print("Is {0} iterable? {1}.".format(sentence, isIterable(sentence)))

Is Sentence(hello) iterable? True.


The `iter` built-in function:

1. Checks whether the object implements `__iter__`, and calls that to obtain an iterator.
2. If `__iter__` is not implemented, but `__getitem__` is implemented, Python creates an iterator
    that attemps to fetch items in order, starting from index 0 (zero).
3. If that fails, Python raises `TypeError`, usually saying "C object is not iterable", where C is 
    the class if the target object.

#### 迭代器

> An iterator is an object with a `__next__` and `__iter__` method.
> 迭代器是定义了`__next__` 和`__iter__`方法的对象。

The standard interface for an iterator has two methods:

* `__next__`: Returns the next available item, raising `StopIteration` when there are no more items.
* `__iter__`: Returns `self`; this allows iterators to be used where an iterable is expected, for example, in a `for` loop.

![](iterable_iterator.png)

Since iterators also implement the` __iter__` method, they are iterable as well.

In [26]:
# The __iter__ method of Sentence returns a SentenceIterator.
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = re.findall(r'\w+', text)
    def __repr__(self):
        return 'Sentence(%s)' % self.text
    def __iter__(self):
        return SentenceIterator(self.words)

class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
    
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

sentence = Sentence('"The time has come')
for word in sentence :
    print(word)

The
time
has
come


The best way to check if an object `x` is an iterator is to call `isinstance(x, abc.Iterator)`.

In [29]:
from collections import abc
isinstance(sentence.__iter__(), abc.Iterator)

True

!!! note
    可迭代对象不能把自己作为迭代器。也就是说，可迭代对象有实现`__next__`方法。这遵循了[迭代器设计模式](../../Java/Head First设计模式/9 Iterator and Composite Pattern.md)：如果可迭代对象实现了`__next__`，它不能满足多次迭代的需要。正确的做法是将可迭代对象和迭代器分离。

#### 生成器

And Python function that has the `yield` keyword in its body is a **generator function**(生成器函数): a function which called, return a **generator**(生成器) object.

In [ ]:
def gen_123():
    yield 1
    yield 2
    yield 3
print(gen_123)   # gen_123是个函数： <function gen_123 at 0x107b03b70>
print(gen_123())  # gen_123返回生成器：<generator object gen_123 at 0x107aea408>

The `__iter__` method below is a generator function which, when called, builds a generator object that implements the iterator interface, So the `SentenceIterator` class is no longer needed.

In [ ]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = re.findall(r'\w+', text)
    def __repr__(self, text):
        return 'Sentence(%s)' % text
    def __iter__(self):
        for word in self.words:
            yield word

A **generator expression**(生成器表达式) can be understood as a lazy version of a list comprehension: It does not eagerly build a list, but returns a generator that will lazily produce the items on demand.

In [31]:
l = [1, 2, 3, 4]
(l*2 for item in l)  # The generator expression produces a generator
#<generator object <genexpr> at 0x107a4bb10>

<generator object <genexpr> at 0x107a4bb10>

In [33]:
# Sentence uses a generator expression in its __iter__ method
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = re.finditer(r'\w+', text)  # lazy 
    def __repr__(self, text):
        return 'Sentence(%s)' % text
    def __iter__(self):
        return (match.group() for match in self.words) # generator expression
for word in Sentence("hello world!"):
    print(word)

hello
world
